# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tura,25.5198,90.2201,28.01,73,2,2.59,IN,1694660161
1,1,yellowknife,62.4560,-114.3525,15.31,83,100,2.57,CA,1694660132
2,2,santa maria,-29.6842,-53.8069,10.61,100,100,2.57,BR,1694660162
3,3,carnarvon,-24.8667,113.6333,21.33,57,0,3.10,AU,1694660162
4,4,albany,42.6001,-73.9662,18.36,93,2,2.17,US,1694660162


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM")

                     
# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values

filtered_cities_df=city_data_df.loc[(city_data_df["Max Temp"]>21)
                                   & (city_data_df["Max Temp"]<27)
                                   & (city_data_df["Cloudiness"]== 0)
                                   & (city_data_df["Wind Speed"]>4.5)
                                   & (city_data_df["Humidity"]<80)]

# Drop any rows with null values
filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
173,173,mirabad,30.4362,61.8383,25.99,24,0,10.33,AF,1694659983
271,271,yeppoon,-23.1333,150.7333,25.88,60,0,6.40,AU,1694660228
277,277,guerrero negro,27.9769,-114.0611,21.92,75,0,7.50,MX,1694659768
408,408,ghadamis,30.1337,9.5007,25.80,20,0,4.59,LY,1694660261
567,567,serra talhada,-7.9919,-38.2983,21.50,79,0,6.01,BR,1694660301


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=filtered_cities_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
173,mirabad,AF,30.4362,61.8383,24,
271,yeppoon,AU,-23.1333,150.7333,60,
277,guerrero negro,MX,27.9769,-114.0611,75,
408,ghadamis,LY,30.1337,9.5007,20,
567,serra talhada,BR,-7.9919,-38.2983,79,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
              "apiKey":geoapify_key}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
   
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
mirabad - nearest hotel: No hotel found
yeppoon - nearest hotel: Blossom's on Seaspray
guerrero negro - nearest hotel: Plaza sal paraiso
ghadamis - nearest hotel: فندق عين الفارس
serra talhada - nearest hotel: Hotel do Vale


,City,Country,Lat,Lng,Humidity,Hotel Name
173,mirabad,AF,30.4362,61.8383,24,No hotel found
271,yeppoon,AU,-23.1333,150.7333,60,Blossom's on Seaspray
277,guerrero negro,MX,27.9769,-114.0611,75,Plaza sal paraiso
408,ghadamis,LY,30.1337,9.5007,20,فندق عين الفارس
567,serra talhada,BR,-7.9919,-38.2983,79,Hotel do Vale


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
city_hotel_map = hotel_df.hvplot.points("Lng",
                                        "Lat",
                                     geo=True,
                                        color="City",
                                      alpha=1,
                            size = "Humidity",
                                tiles = "OSM",
          hover_cols=["Hotel Name","Country"],
                            frame_width = 700,
                           frame_height = 700)
                                     

# Display the map
city_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)